<a href="https://colab.research.google.com/github/gangula-karthik/Altitude-Analytics/blob/main/ALTITUDE_ANALYTICS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'training-data:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4396626%2F7549185%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240211%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240211T071801Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D86ac5ac0f44e23ae4c00f8dc1b74975f1c16f21f4bb22708cb8acf6f103438f012f47ad2ef24dc7d71b298a5af2676cc12c502c95f76ffd8e0395ccfae19ce32e3a7af07f66a34a1442ca2baeb29675b968ede2fd31129b45cc0d3257d82baf279e910f60721d8333318600115c691e984827c2ffdde92e7d67f23a530f3611896bd7c316fb96e2514f161b5455536dfaaa56080e520c948bbf5219b2332d43b1b88de266333b6e864d1f2bbb437ea6cb37c1f7ba92fd8a8d7a00d7da054f68161eb38787d634c71896f2ef8fa9bc8e381c0e5644ed08e5683b180e98c908bc8439f11dafaf96c44773e0d817dc11bdedfc821a3b7f694282fdefbf227adc385,testing-data:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4401450%2F7558364%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240211%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240211T071801Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D1d11ab953466783519353f1ec18e0e2b96e57740e8b53e8b588418864a690752d31df9b4a015089448f8c64d127ea4f49334dbe25a23450f3a940441af5e80f3054b89fe79100234de936e3afe1e974d1ede1d7eb46ffb2a14b39b7e0788264d969ce4c4572c83a847dccc163713357dd8663089881104d6acee8dd2290f3eae1d264d473f07c0f40ea1083901946a7f73173f296a60a09293e10262f354059a58e6bc8d8ac188f27bd7298ad83890cc4abc308d614fa2c84359fcd99f86c79e7e12f5fae9bc6964132b74f539c89ff6fdfbe5319bda0f1a213f68d06899d6b260f55f5db827cd5dac7df8370ec9d9b3fcccfc3783b8338a3cafd050f6301288,dataprocessing:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4401471%2F7558406%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240211%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240211T071801Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Daca257a4ff2619306ae1f3090e5cd238ee2ebd302093af305ac508c74d67aace667ebd765be6db7e0149ceb949fdf6d1cc12dafb9af7ce4f80101421f0aed2c64810791c5cdb52d78734bf65bcff1f5c66b544715b9a0a246978068893b3717f85c7ffcda75c0d48b0cf8e5d8366632687a73cf6048a3b803a200fb7a462e4c40e6e553c2e1a1f46f660994d5cbe6807d193fdb7bb7f863e42cfe5cacca3a3b35833b44c2c86019532088abb04852f1f4961e1094ec2a7fffb34047cb64de386cb808c3d23ef5f6b6ef36b8dee599710db85dad0d654c813a71fc7cdc994c4e7ccd85d45e97f8ccad76a3de3e2032c74f1c9955f439476f81ec62341f46d61ca'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


# NPS SCORE CLASSIFICATION ON AIRLINES REVIEW TEXT

## Installing and Importing Packages

In [ ]:
!nvidia-smi # checking for GPU ver.

/bin/bash: nvidia-smi: command not found


In [ ]:
# %pip install imbalanced-learn
# %pip install lightgbm
# %pip install cuml
%pip install contractions
%pip install symspellpy
%pip install textstat
%pip install -U kaleido
# %pip show cuml

# %pip install gensim==3.4.0
# %pip install smart_open==1.9.0

import nltk
# nltk.download('wordnet', '/usr/share/nltk_data')
# nltk.download()
# !unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 7.4 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 5.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 34.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 kB 2.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for editdistpy: filename=editdistpy-0.1.3-cp310-cp310-linux_x86_64.whl size=50410 sha256=89ebee0f0f8395b5c29a99f58f00658efa770a5cf62b77aeb09a18a02e0f482a
  Stored in directory: /root/.cache/pip/wheels/88/6a/a6/a1283cc145323a1fb3d475bd158ee60b248ab1985230d266fc
Successfully built editdistpy
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 2.9 MB/s eta 0:00:00
     ━━━━

In [ ]:
import string
import warnings
import re
import sys
sys.path.append('/kaggle/input/dataprocessing/')

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import spacy
from bs4 import BeautifulSoup
from symspellpy import SymSpell
from textstat import textstat
from pprint import pprint
from collections import Counter
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN
import nltk
from nltk import word_tokenize, pos_tag, sent_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize, pos_tag, pos_tag_sents
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import (accuracy_score, classification_report,
                             confusion_matrix, f1_score,
                             precision_recall_fscore_support,
                             precision_score, recall_score)
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import ComplementNB, MultinomialNB
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer, LabelEncoder
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import loguniform
from tqdm import tqdm
from wordcloud import WordCloud
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
import contractions
from scipy.stats import loguniform, uniform
from scipy.sparse import hstack, vstack
from sklearn.base import BaseEstimator, TransformerMixin

from emojii_and_emoticon_map import EMOTICONS_EMO, EMOJI_UNICODE
# from DataScraping_and_processing.emojii_and_emoticon_map import EMOTICONS_EMO, EMOJI_UNICODE
import langid

In [ ]:
import gensim.corpora as corpora
from gensim.models import LdaMulticore
from gensim.models import LdaModel
from gensim.models import FastText

In [ ]:
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

## Data Preparation + Feature Engineering

### Simple Data Cleaning
- reading the csv file
- removing null values (appears after data cleaning)
- Label encoding on NPS_category `[mapping: {"Detractor": 0, "Neutral": 1, "Promoter": 2}]`

In [ ]:
df = pd.read_csv("/kaggle/input/training-data/karthik_cleaned_data.csv", index_col=[0])
# df = pd.read_csv("./DataScraping_and_processing/karthik_cleaned_data.csv", index_col=[0])
df.head()

,overall_rating,review_date,review_text,airline_name,NPS_category,NPS,language_info,clean_text,text_length,word_count,unique_word_count,word_density,uppercase_words,comma_count,exclamation_count,question_mark_count,avg_sentence_length,flesch_reading_score,gunning_fog_index
0,Rated 1 out of 5 stars,2023-09-18 03:29:15,EasyJet sent text at 4.00 am day of flight hom...,easyjet,Detractor,-1,en,text rush organise transport effort implicatio...,194,32,29,5.878788,0,1,0,0,6.600000,81.90,5.06
1,9,2023-12-01 00:00:00,Its been a few years when I flew a lot in A...,Vistara,Promoter,1,en,asia vistara surprise swift immaculate steward...,608,109,74,5.527273,3,7,0,0,27.250000,61.19,14.08
3,Rated 1 out of 5 stars,2018-09-09 20:37:11,one of the worst experiences with Air France e...,air france,Detractor,-1,en,ever bore carry duty inbound,204,30,30,6.580645,0,2,0,0,10.000000,61.33,8.00
4,Rated 1 out of 5 stars,2019-12-23 14:00:19,Not a single star this airlines deserves .I lo...,egyptair,Detractor,-1,en,deserves last block respond mail operator harr...,464,87,64,5.272727,4,2,0,0,14.833333,74.39,7.45
5,Rated 1 out of 5 stars,2023-12-29 06:42:56,I was forced to pay 150-euro worth penalty for...,ryanair,Detractor,-1,en,penalty fail earth would dare season arrogant ...,240,38,35,6.153846,1,1,0,1,12.666667,58.58,10.34


In [ ]:
df.isna().sum()

overall_rating            0
review_date               0
review_text               0
airline_name              0
NPS_category              0
NPS                       0
language_info             0
clean_text              719
text_length               0
word_count                0
unique_word_count         0
word_density              0
uppercase_words           0
comma_count               0
exclamation_count         0
question_mark_count       0
avg_sentence_length       0
flesch_reading_score      0
gunning_fog_index         0
dtype: int64

<div class="alert alert-info" role="alert">
    There are null values for clean text. This is because of the remove frequent words and rare terms function which has removed all the words from certain reviews. Need to further adjust the term frequency threshold to prevent removal of many words. These functions must be updated. The list of stopwords have also been udpated to include words like "u", "never" and "could" which appeared quite often when doing topic modelling.
</div>

In [ ]:
slang_dict = {}
word_freq = Counter()

with open('/kaggle/input/dataprocessing/slang.txt', 'r') as file:
    slang_dict = {key.lower(): value.lower() for line in file if '=' in line for key, value in [line.strip().split('=', 1)]}

# with open('DataScraping_and_processing/slang.txt', 'r') as file:
#     slang_dict = {key.lower(): value.lower() for line in file if '=' in line for key, value in [line.strip().split('=', 1)]}

tfidf_vectorizer = TfidfVectorizer(stop_words='english', min_df=2)
tfidf_matrix = tfidf_vectorizer.fit_transform(df['review_text'])

feature_names = np.array(tfidf_vectorizer.get_feature_names_out())

tfidf_sum = np.sum(tfidf_matrix, axis=0)
tfidf_scores = np.squeeze(np.asarray(tfidf_sum))

sorted_indices = np.argsort(tfidf_scores)

freq_word_threshold = np.quantile(tfidf_scores, 0.99)
rare_word_threshold = np.quantile(tfidf_scores, 0.01)

FREQWORDS = set(feature_names[tfidf_scores > freq_word_threshold])
RAREWORDS = set(feature_names[tfidf_scores < rare_word_threshold])

print("Frequent Words:", list(FREQWORDS)[:5])
print("\nRare Words:", list(RAREWORDS)[:5])

def detect_language(text):
    lang, _ = langid.classify(text)
    return lang

tqdm.pandas(desc="Detecting languages")
df['language_info'] = df['review_text'].progress_apply(detect_language)

def remove_rows(dataframe):
    print("rows removed")
    return (dataframe.dropna()
            .drop_duplicates()
            .query("language_info == 'en'")
            .loc[dataframe['review_text'].str.split().str.len() > 2])

data_cleaning_pipeline = Pipeline([
    ('remove_rows', FunctionTransformer(remove_rows)),
])

# text cleaning functions

def remove_excess_space(text):
    text = " ".join(text.split())
    text = re.sub(r'\s([?.!"](?:\s|$))', r'\1', text)
    return text

def remove_url(text):
    regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    return re.sub(regex, '', text)

def html_tag_remover(text):
    soup = BeautifulSoup(text, "html.parser")
    text = soup.get_text()
    return text

def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

def normalize_exaggerated_text(text):
    normalized_text = re.sub(r'(.)\1{2,}', r'\1', text)
    return normalized_text

def replace_slang(text):
    pattern = re.compile(r'\b(' + '|'.join(re.escape(key) for key in slang_dict.keys()) + r')\b')
    return pattern.sub(lambda x: slang_dict[x.group().lower()], text)

def convert_emoticons_and_emojis(text, emoticons_emo=EMOTICONS_EMO, emoji_unicode=EMOJI_UNICODE):
    emoticon_patterns = {re.escape(k): "_".join(v.replace(",", "").split()) for k, v in emoticons_emo.items()}
    emoji_patterns = {re.escape(v): k.strip(":") for k, v in emoji_unicode.items()}
    combined_patterns = {**emoticon_patterns, **emoji_patterns}

    pattern = re.compile('|'.join(combined_patterns.keys()))

    def replace(match):
        return combined_patterns[re.escape(match.group(0))]

    return pattern.sub(replace, text)

def remove_username(text):
    return re.sub('@[^\s]+','',text)

symsp = SymSpell()
symsp.load_dictionary('/kaggle/input/dataprocessing/my_frequency_dictionary.txt', term_index=0, count_index=1, separator=' ')

def correct_spelling(text):
    corrected = symsp.lookup_compound(text, max_edit_distance=2)
    return corrected[0].term if corrected else text

def remove_money(text):
    pattern = r'\b(?:\$|\£|\€|\₹)?\w*\d\w*\b'
    return re.sub(pattern, '', text)

def fix_negation(sentence):
    for i in range(1, len(sentence)):
        if sentence[i-1] in ['not', "n't"]:
            synsets = wordnet.synsets(sentence[i])
            if synsets:
                w1 = synsets[0]
                antonyms = [l.antonyms()[0].name() for l in w1.lemmas() if l.antonyms()]
                if antonyms:
                    max_dissimilarity = 0
                    for ant in antonyms:
                        ant_synsets = wordnet.synsets(ant)
                        if ant_synsets:
                            similarity = w1.wup_similarity(ant_synsets[0])
                            # Check if similarity is None and handle it
                            if similarity is None:
                                continue  # Skip this comparison
                            dissimilarity = 1 - similarity
                            if dissimilarity > max_dissimilarity:
                                max_dissimilarity = dissimilarity
                                sentence[i] = ant
                    sentence[i-1] = ''

    sentence = [word for word in sentence if word]
    return ' '.join(sentence)  # Return a string instead of a list


STOPWORDS = stopwords.words("english")
STOPWORDS += ['u', 'us', 'back', 'one', 'never', 'could']

def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

def remove_freqwords(text):
    return " ".join([word for word in str(text).split() if word not in FREQWORDS])

def remove_rarewords(text):
    return " ".join([word for word in str(text).split() if word not in RAREWORDS])

def lemmatize_words(text):
    lemmatizer = WordNetLemmatizer()
    wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}
    pos_tagged_text = nltk.pos_tag(text.split())
    return " ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])


tqdm.pandas()

text_preprocessing_pipeline = Pipeline(steps=[
    ('lowercase', FunctionTransformer(lambda x: x.progress_apply(str.lower))),
    ('remove excess whitespace', FunctionTransformer(lambda x: x.progress_apply(remove_excess_space))),
    ('remove_url', FunctionTransformer(lambda x: x.progress_apply(remove_url))),
    ('html_tag_remover', FunctionTransformer(lambda x: x.progress_apply(html_tag_remover))),
    ('money_remover', FunctionTransformer(lambda x: x.progress_apply(remove_money))),
    ('convert_emoticons_and_emojii', FunctionTransformer(lambda x: x.progress_apply(convert_emoticons_and_emojis))),
    ('remove_username', FunctionTransformer(lambda x: x.progress_apply(remove_username))),
    ('replace_slang', FunctionTransformer(lambda x: x.progress_apply(replace_slang))),
    ('correct_spelling', FunctionTransformer(lambda x: x.progress_apply(correct_spelling))),
    ('expand_contractions', FunctionTransformer(lambda x: x.progress_apply(lambda text: contractions.fix(text)))),
    ('fix_negation', FunctionTransformer(lambda x: x.progress_apply(lambda text: "".join(fix_negation(word_tokenize(text)))))),
    ('remove_punctuation', FunctionTransformer(lambda x: x.progress_apply(remove_punctuation))),
    ('normalize_exaggerated_text', FunctionTransformer(lambda x: x.progress_apply(normalize_exaggerated_text))),
    ('remove_stopwords', FunctionTransformer(lambda x: x.progress_apply(remove_stopwords))),
    ('remove_freqwords', FunctionTransformer(lambda x: x.progress_apply(remove_freqwords))),
    ('remove_rarewords', FunctionTransformer(lambda x: x.progress_apply(remove_rarewords))),
    ('lemmatize_words', FunctionTransformer(lambda x: x.progress_apply(lemmatize_words)))
])

Frequent Words: ['cheap', 'poor', 'right', 'plane', 'half']

Rare Words: ['forsøg', 'sliema', 'automized', '35000', 'medida']


Detecting languages: 100%|██████████| 57325/57325 [04:36<00:00, 207.27it/s]


In [ ]:

df = data_cleaning_pipeline.fit_transform(df)
df['clean_text'] = text_preprocessing_pipeline.fit_transform(df['review_text'])
df.head()

rows removed


  0%|          | 1/56606 [00:00<32:30, 29.02it/s]


LookupError: 
**********************************************************************
  Resource 'corpora/wordnet' not found.  Please use the NLTK
  Downloader to obtain the resource:  >>> nltk.download()
  Searched in:
    - '/root/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************

In [ ]:
df["NPS_category"] = df["NPS_category"].map({"Detractor": 0, "Neutral": 1, "Promoter": 2})
df.dropna(inplace=True)
df.to_csv("cleaned_dataset_v2.csv")

In [ ]:
df.describe().T

### Adding Meta text features
- meta text features are those additional features about the text that can be used to give more information about it. These can include things like word count, count of nouns and verbs, text readability score, etc

In [ ]:
X = df[['text_length', 'word_count', 'unique_word_count', 'word_density', 'uppercase_words', 'comma_count', 'exclamation_count', 'question_mark_count', 'avg_sentence_length', 'flesch_reading_score', 'gunning_fog_index']]
y = df['NPS_category']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=42)

rf_classifier = RandomForestClassifier(class_weight='balanced', n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

y_pred = rf_classifier.predict(X_valid)
accuracy = accuracy_score(y_valid, y_pred)
precision = precision_score(y_valid, y_pred, average='weighted')
recall = recall_score(y_valid, y_pred, average='weighted')
f1 = f1_score(y_valid, y_pred, average='weighted')
print("Accuracy:", accuracy)
print("Precision (Weighted):", precision)
print("Recall (Weighted):", recall)
print("F1-score (Weighted):", f1)

# Get feature importance
feature_importance = rf_classifier.feature_importances_
feature_importance_df = pd.DataFrame({'Features': X.columns, 'Importance': feature_importance})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Plot the feature importance
plt.figure(figsize=(10, 6))
sns.barplot(data=feature_importance_df, x='Importance', y='Features', orient="h")
plt.xlabel("Importance")
plt.ylabel("Features")
plt.title("Feature Importance")
plt.show()

<div class="alert alert-info" role="alert">
    The features created are not very strong at predicting the target. Upon further research some more stronger features included the noun count and the verb count.
</div>

In [ ]:
# calculating the nouns and verbs

def pos_proportions(text, pos_tag):
    sentences = sent_tokenize(text)
    tokenized_sentences = [word_tokenize(sentence) for sentence in sentences]
    tagged_sentences = pos_tag_sents(tokenized_sentences)

    pos_counts = 0
    total_words = 0

    for sentence in tagged_sentences:
        for word, tag in sentence:
            if word.isalpha():
                total_words += 1
                if tag.startswith(pos_tag):
                    pos_counts += 1

    return pos_counts / total_words if total_words > 0 else 0

tqdm.pandas(desc="Counting the nouns")
df["noun_proportions"] = df['review_text'].progress_apply(lambda x: pos_proportions(x, "NN"))
tqdm.pandas(desc="Counting the verbs")
df["verb_proportions"] = df['review_text'].progress_apply(lambda x: pos_proportions(x, "VB"))
df[["clean_text", "noun_proportions", "verb_proportions"]]

In [ ]:
X = X.assign(noun_proportions=df['noun_proportions'], verb_proportions=df['verb_proportions'])
y = df['NPS_category']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

rf_classifier = RandomForestClassifier(class_weight='balanced', n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

y_pred = rf_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision (Weighted):", precision)
print("Recall (Weighted):", recall)
print("F1-score (Weighted):", f1)

feature_importance = rf_classifier.feature_importances_
feature_importance_df = pd.DataFrame({'Features': X.columns, 'Importance': feature_importance})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(data=feature_importance_df, x='Importance', y='Features', orient="h")
plt.xlabel("Importance")
plt.ylabel("Features")
plt.title("Feature Importance")
plt.show()

In [ ]:
threshold = 0.06 # threshold for removing features

feature_importance_df = pd.DataFrame({'Features': X.columns, 'Importance': feature_importance})

feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Identify features with importance below the threshold
low_importance_features = feature_importance_df[feature_importance_df['Importance'] < threshold]['Features']

X_reduced = X.drop(columns=low_importance_features)
X_reduced

In [ ]:
# removing the redudandant columns
columns_to_remove = ['word_count', 'comma_count', 'flesch_reading_score']

# Remove the specified columns
X_reduced = X_reduced.drop(columns=columns_to_remove, errors='ignore')
X_reduced

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_reduced = scaler.fit_transform(X_reduced)
X_reduced

In [ ]:
X_reduced_train, X_reduced_valid = train_test_split(X_reduced, test_size=0.25, random_state=42)

### Bag of words ❌

- Bag of words was used for its simplicity and ease of use
- Effective for short to medium texts with a mean word count of 121
- Serves as a good baseline
- Experimented with:
    - Adding meta text features
    - Bigrams

In [ ]:
# creating bag of words representation

count_vectorizer = CountVectorizer()
bow_matrix = count_vectorizer.fit_transform(df["clean_text"])

X_train_bow, X_valid_bow, y_train_bow, y_valid_bow = train_test_split(
    bow_matrix, df["NPS_category"], test_size=0.25, random_state=42)

clf_bow = LogisticRegression(class_weight='balanced', max_iter=1500)
clf_bow.fit(X_train_bow, y_train_bow)
y_pred_bow = clf_bow.predict(X_valid_bow)
print("BoW Classification Report:\n", classification_report(y_valid_bow, y_pred_bow))

In [ ]:
# X_reduced_train, X_reduced_valid

X_train_combined = hstack([X_train_bow, X_reduced_train])
X_valid_combined = hstack([X_valid_bow, X_reduced_valid])

clf_combined = LogisticRegression(class_weight='balanced', max_iter=1500)
clf_combined.fit(X_train_combined, y_train)

y_pred_combined = clf_combined.predict(X_valid_combined)
print(classification_report(y_valid_bow, y_pred_combined))

<div class="alert alert-info" role="alert">
--> Relatively stable performance with a very slight increase in precision (for class 2), recall (for class 0), and f1-score (for all classes). <br>
--> Indicates that combing bag of words with meta features shows no performance boost. <br>
Lets see if ngrams is effective in observing performance gains.
</div>

In [ ]:
# creating bag of words representation with unigrams + bigrams
count_vectorizer = CountVectorizer(ngram_range=(1, 2))
bow_matrix = count_vectorizer.fit_transform(df["clean_text"])

X_train_bow, X_valid_bow, y_train_bow, y_valid_bow = train_test_split(
    bow_matrix, df["NPS_category"], test_size=0.25, random_state=42)

clf_bow = LogisticRegression(class_weight='balanced', max_iter=1500)
clf_bow.fit(X_train_bow, y_train_bow)
y_pred_bow = clf_bow.predict(X_valid_bow)
print("BoW Classification Report:\n", classification_report(y_valid_bow, y_pred_bow))

<div class="alert alert-info" role="alert">
    --> For Class 0, the inclusion of bigrams led to an improvement in both precision (from 0.96 to 0.95) and recall (from 0.87 to 0.93) for Class 0 which suggests that the bigrams were slightly effective in capturing the contextual information for this class <br><br>
    --> For Class 1, there was a slight increase in precision (from 0.08 to 0.12) but a decrease in recall (from 0.19 to 0.09). This indicates that while the model became slightly more precise in its predictions for Class 1 with bigrams, it also became less sensitive to detecting true instances of this class. This could be due to the very large class imbalance and the fact that the class weights are not effective enough at handling it.<br><br>
    --> For Class 2, the addition of bigrams resulted in an increase in both precision (from 0.56 to 0.62) and recall (from 0.75 to 0.75) for Class 2. This improvement suggests that bigrams provided valuable context that enhanced the model's ability to accurately classify instances of Class 2.
<br><br>
We will not try trigrams since the similar performance is to be expected and also because of the long runtime.
</div>

### TFIDF ❌
- TF-IDF improves upon the basic Bag of Words model by not just counting words, but by assigning weights to words based on their importance. Common words across all documents receive lower weights, while unique words in specific documents are emphasized (reduction in noise)
- Can better capture the relevance of terms within specific reviews. This is crucial in understanding the sentiment and specifics of airline service, where certain terms might be particularly indicative of positive or negative experiences.
- Experimented with:
    - Adding meta text features
    - Bigrams

In [ ]:
# creating a TF-IDF text representation

tfidf_vectorizer = TfidfVectorizer()

tfidf_matrix = tfidf_vectorizer.fit_transform(df["clean_text"])

X_train_tfidf, X_valid_tfidf, y_train_tfidf, y_valid_tfidf = train_test_split(
    tfidf_matrix, df["NPS_category"], test_size=0.25, random_state=42)

clf_tfidf = LogisticRegression(class_weight='balanced', max_iter=2000)
clf_tfidf.fit(X_train_tfidf, y_train_tfidf)
y_pred_tfidf = clf_tfidf.predict(X_valid_tfidf)
print("TF-IDF Classification Report:\n", classification_report(y_valid_tfidf, y_pred_tfidf))

<div class="alert alert-info" role="alert">
--> Performance wise is already much higher than the bag of words embeddings, especially for the passives/neutral/class 1 in terms of recall. There is a slight dip in precision (by 0.03 to 0.04) for class 1 and 2
</div>

In [ ]:
# X_reduced_train, X_reduced_valid

X_train_combined = hstack([X_train_tfidf, X_reduced_train])
X_valid_combined = hstack([X_valid_tfidf, X_reduced_valid])

clf_combined = LogisticRegression(class_weight='balanced', max_iter=2000)
clf_combined.fit(X_train_combined, y_train_tfidf)

y_pred_combined = clf_combined.predict(X_valid_combined)
print(classification_report(y_valid_tfidf, y_pred_combined))

<div class="alert alert-info" role="alert">
--> No performance gain can be seen in any class when adding the text meta features
</div>

In [ ]:
# creating tfidf representation with unigrams + bigrams
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2))
tfidf_matrix = tfidf_vectorizer.fit_transform(df["clean_text"])

X_train_tfidf, X_valid_tfidf, y_train_tfidf, y_valid_tfidf = train_test_split(
    tfidf_matrix, df["NPS_category"], test_size=0.25, random_state=42)


clf_tfidf = LogisticRegression(class_weight='balanced', max_iter=2000)
clf_tfidf.fit(X_train_tfidf, y_train_tfidf)
y_pred_tfidf = clf_tfidf.predict(X_valid_tfidf)
print("TF-IDF Classification Report:\n", classification_report(y_valid_tfidf, y_pred_tfidf))

<div class="alert alert-info" role="alert">
--> After adding in bigrams the model is able to predict class 2 better than before, however there is a significant drop in recall for class 1 which is of concern. <br>
--> Since all the classes are equally important, tfidf on its own is not very suitable. A new technique has been research and found called delta tfidf which will be explore below.
</div>

### Delta Tfidf ✅

- Enhanced Feature Discrimination: Delta TF-IDF amplifies the features (words) that are more discriminative for each class by adjusting their TF-IDF values based on their deviation from a mean IDF calculated across all classes leading to better separation of classes.
- However, one thing to note is that delta TF-IDF has negative values which means it cannot be used with naive bayes models. An alternative is to use Gaussian Bayes, however it doesnt support class weights, which makes it unsuitable to use.

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(df['clean_text'], df['NPS_category'], test_size=0.25, random_state=42)

tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

class_idfs = []
unique_classes = np.unique(y_train)
for class_idx in unique_classes:
    class_mask = (y_train == class_idx)
    class_docs = X_train[class_mask]
    class_vectorizer = TfidfVectorizer(vocabulary=tfidf_vectorizer.vocabulary_)
    class_tfidf = class_vectorizer.fit_transform(class_docs)
    class_idf = class_vectorizer.idf_
    class_idfs.append(class_idf)

mean_idf = np.mean(class_idfs, axis=0)

delta_idfs = [class_idf - mean_idf for class_idf in class_idfs]

class_to_index = {label: index for index, label in enumerate(unique_classes)}

X_train_delta_tfidf = []
for i, doc in enumerate(X_train_tfidf):
    class_idx = class_to_index[y_train.iloc[i]]
    delta_idf = delta_idfs[class_idx]
    X_train_delta_tfidf.append(doc.multiply(delta_idf))

X_train_delta_tfidf = vstack(X_train_delta_tfidf)

X_valid_tfidf = tfidf_vectorizer.transform(X_valid)
X_valid_delta_tfidf = []
for i, doc in enumerate(X_valid_tfidf):
    class_idx = class_to_index[y_valid.iloc[i]]
    delta_idf = delta_idfs[class_idx]
    X_valid_delta_tfidf.append(doc.multiply(delta_idf))

X_valid_delta_tfidf = vstack(X_valid_delta_tfidf)

clf = LogisticRegression(class_weight='balanced')
clf.fit(X_train_delta_tfidf, y_train)

y_pred = clf.predict(X_valid_delta_tfidf)
print(classification_report(y_valid, y_pred))

<div class="alert alert-info" role="alert">
--> A much signficant performance upgrade can be seen across all the classes. Since adding meta features and bigrams didnt make much of a difference for tfidf, it is very likely that it will not make much difference for delta tfidf as well.
</div>

## Data Balancing Techniques
Dataset is highly imbalanced which means that the predictions will be highly biased towards the majority class. To fix this it is important to balance the dataset through:

- using class weights
- using SMOTE
- using adasyn

*note: random oversampling was not used since it works by duplicating the samples in the minority class, even if it has better performance, it is most likely overfitting.*

### Delta Tfidf + class weights (Balancing Technique) ✅

In [ ]:
clf = LogisticRegression(class_weight='balanced')

clf.fit(X_train_delta_tfidf, y_train)
y_pred = clf.predict(X_valid_delta_tfidf)

print(classification_report(y_valid, y_pred))

### Delta Tfidf + SMOTE (Balancing Technique) ❌
- should not be applied on the test data to preserve the original distribution.

In [ ]:
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_delta_tfidf, y_train)

clf_smote = LogisticRegression()
clf_smote.fit(X_train_smote, y_train_smote)
y_pred = clf_smote.predict(X_valid_delta_tfidf)

print("Classification Report after SMOTE:\n", classification_report(y_valid, y_pred))

<div class="alert alert-info" role="alert">
--> A low precision (high false positives) is seen for class 1, this shows that SMOTE is not the best technique to apply.
</div>
</div>

### Delta Tfidf + Adasyn (Balancing Technique) ❌
- should not be applied on the test data to preserve the original distribution.

In [ ]:
adasyn = ADASYN(random_state=42)
X_train_adasyn, y_train_adasyn = adasyn.fit_resample(X_train_delta_tfidf, y_train)

clf_adasyn = LogisticRegression()
clf_adasyn.fit(X_train_adasyn, y_train_adasyn)
y_pred = clf_adasyn.predict(X_valid_delta_tfidf)

print("Classification Report after ADASYN:\n", classification_report(y_valid, y_pred))

<div class="alert alert-info" role="alert">
--> A low precision (high false positives) is seen for class 1, this shows that ADASYN is not the best technique to apply. Class weighting will be the selected technique. This means that for modelling it is important to utilise models that support adding weights for the classes.
</div>
</div>

## Modelling

Some considerations when building models:

- The SVM classifier can be built using LinearSVC and Stochastic gradient descent with hinge loss. LinearSVC uses the full data to solve the optimization problem which makes it suitable on small datasets however, on larger and more sparse data, SGD (with hinge loss) is more suitable as it handles data in batches assuming data points are independent and identically distributed.

- Boosting algorithms comparison
    - for Adaboost, 2 different stump learners was used which was decision tree (most commonly used) and logistic regression (better baseline performance than decision tree).
    - LightGBM, Catboost and XGBoost is used due to its fast performance and large number of tunable parameters.
    
<div style="text-align: center;">
    <img src="https://miro.medium.com/v2/resize:fit:1100/format:webp/1*0xWny_DiMZxN3Ajpg6G5tg.png" style="width: 800px; height: 400px;">
</div>

In [ ]:
models = {
    "Logistic Regression": LogisticRegression(multi_class='multinomial', solver='lbfgs', class_weight='balanced', max_iter=2000),
    "Decision Tree": DecisionTreeClassifier(class_weight='balanced'),
    "SVM (SGD)": SGDClassifier(loss='hinge', class_weight='balanced', max_iter=1000, tol=1e-3),
    "Random Forest": RandomForestClassifier(class_weight='balanced'),
    "XGBoost": XGBClassifier(enable_categorical=True, eval_metric='mlogloss'),
    "AdaBoost+DT": AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1), n_estimators=100),
    "AdaBoost+LR": AdaBoostClassifier(base_estimator=LogisticRegression(multi_class='multinomial',class_weight='balanced',max_iter=1000),n_estimators=100),
#     "Catboost": CatBoostClassifier(iterations=500,learning_rate=0.1,depth=6,l2_leaf_reg=3,cat_features=[], auto_class_weights='Balanced',verbose=200, task_type='GPU',devices='0:1'),
#     "LightGBM": LGBMClassifier(boosting_type='gbdt', objective='multiclass', class_weight='balanced', learning_rate=0.09, max_depth=-1, random_state=42, n_estimators=100, device='gpu', gpu_platform_id=0, gpu_device_id=0)
}

def evaluate_model(name, model, X_train, y_train, X_test, y_test):
    if name == "XGBoost":
        sample_weights_data = compute_sample_weight("balanced", y_train)
        model.fit(X_train, y_train, sample_weight=sample_weights_data)
    else:
        model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    print(f"CLASSIFICATION REPORT FOR {name.upper()}")
    print(classification_report(y_test, y_pred))
    return {"Model": name, "Accuracy": accuracy, "Precision": precision, "Recall": recall, "F1 Score": f1}

In [ ]:
results = pd.DataFrame()

for name, model in tqdm(models.items(), desc="Evaluating Models"):
    results_row = evaluate_model(name, model, X_train_delta_tfidf, y_train, X_valid_delta_tfidf, y_valid)
    results = results._append(results_row, ignore_index=True)

results

<div class="alert alert-info" role="alert">
--> SGD (SVM) and Random Forest exhibit the highest F1-Score, making them top performers for precise classification. <br>
--> Logistic Regression is close behind, with strong metrics, suggesting it's a reliable choice.<br>
--> Decision Tree shows lower performance, which might indicate it's less suited for this task.<br>
--> XGBoost underperforms in this comparison, hinting at the need for more optimization.<br>
</div>

### Dimensionality reduction techniques

#### SVD

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, classification_report
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


# Step 1: Apply Truncated SVD for feature reduction on CPU
n_components = 50  # Adjust the number of components
n_iter = 10  # Adjust the number of iterations
random_state = 42  # Set a fixed random state for reproducibility
algorithm = 'randomized'  # Experiment with different algorithms

svd = TruncatedSVD(
    n_components=n_components,
    n_iter=n_iter,
    random_state=random_state,
    algorithm=algorithm
)

# Fit and transform on training data
X_train_svd = svd.fit_transform(X_train_delta_tfidf.toarray())

# Transform on test data
X_valid_svd = svd.transform(X_valid_delta_tfidf.toarray())
print("SVD finished")

# Step 2: Logistic Regression with class weights for evaluation
logistic_regression = LogisticRegression(class_weight='balanced')
print("logreg init")

# Fit the model
logistic_regression.fit(X_train_svd, y_train)
print("logreg fit")

# Predictions
y_pred = logistic_regression.predict(X_valid_svd)
print("logreg predict")

# Step 3: Evaluation
print("Accuracy:", accuracy_score(y_valid, y_pred))
print(classification_report(y_valid, y_pred))

# Step 4: Visualize SVD results
plt.scatter(X_train_svd[:, 0], X_train_svd[:, 1], c=y_train, cmap=plt.cm.Paired)
plt.colorbar()
plt.title("SVD Visualization of Training Data")
plt.show()

plt.scatter(X_valid_svd[:, 0], X_valid_svd[:, 1], c=y_test, cmap=plt.cm.Paired)
plt.colorbar()
plt.title("SVD Visualization of Test Data")
plt.show()

## Evaluation

In [ ]:
test_df = pd.read_csv("/kaggle/input/dataprocessing/testing_data.csv", index_col=0)
# test_df = pd.read_csv("./DataScraping_and_processing/testing_data.csv", index_col=0)
test_df.head()

In [ ]:
test_df.shape

In [ ]:
test_df["review_text"].dropna()

In [ ]:
slang_dict = {}
word_freq = Counter()

with open('/kaggle/input/dataprocessing/slang.txt', 'r') as file:
    slang_dict = {key.lower(): value.lower() for line in file if '=' in line for key, value in [line.strip().split('=', 1)]}

# with open('DataScraping_and_processing/slang.txt', 'r') as file:
#     slang_dict = {key.lower(): value.lower() for line in file if '=' in line for key, value in [line.strip().split('=', 1)]}

tfidf_vectorizer = TfidfVectorizer(stop_words='english', min_df=2)
tfidf_matrix = tfidf_vectorizer.fit_transform(df['review_text'])

feature_names = np.array(tfidf_vectorizer.get_feature_names_out())

tfidf_sum = np.sum(tfidf_matrix, axis=0)
tfidf_scores = np.squeeze(np.asarray(tfidf_sum))

sorted_indices = np.argsort(tfidf_scores)

freq_word_threshold = np.quantile(tfidf_scores, 0.99)
rare_word_threshold = np.quantile(tfidf_scores, 0.01)

FREQWORDS = set(feature_names[tfidf_scores > freq_word_threshold])
RAREWORDS = set(feature_names[tfidf_scores < rare_word_threshold])

print("Frequent Words:", list(FREQWORDS)[:5])
print("\nRare Words:", list(RAREWORDS)[:5])

def detect_language(text):
    lang, _ = langid.classify(text)
    return lang

tqdm.pandas(desc="Detecting languages")
test_df['language_info'] = test_df['review_text'].progress_apply(detect_language)
test_df.loc[test_df["language_info"] != "en"]

def remove_rows(dataframe):
    print("rows removed")
    return (dataframe.dropna()
            .drop_duplicates()
            .query("language_info == 'en'")
            .loc[dataframe['review_text'].str.split().str.len() > 2])

data_cleaning_pipeline = Pipeline([
    ('remove_rows', FunctionTransformer(remove_rows)),
])

# text cleaning functions

def remove_excess_space(text):
    text = " ".join(text.split())
    text = re.sub(r'\s([?.!"](?:\s|$))', r'\1', text)
    return text

def remove_url(text):
    regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    return re.sub(regex, '', text)

def html_tag_remover(text):
    soup = BeautifulSoup(text, "html.parser")
    text = soup.get_text()
    return text

def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

def normalize_exaggerated_text(text):
    normalized_text = re.sub(r'(.)\1{2,}', r'\1', text)
    return normalized_text

def replace_slang(text):
    pattern = re.compile(r'\b(' + '|'.join(re.escape(key) for key in slang_dict.keys()) + r')\b')
    return pattern.sub(lambda x: slang_dict[x.group().lower()], text)

def convert_emoticons_and_emojis(text, emoticons_emo=EMOTICONS_EMO, emoji_unicode=EMOJI_UNICODE):
    emoticon_patterns = {re.escape(k): "_".join(v.replace(",", "").split()) for k, v in emoticons_emo.items()}
    emoji_patterns = {re.escape(v): k.strip(":") for k, v in emoji_unicode.items()}
    combined_patterns = {**emoticon_patterns, **emoji_patterns}

    pattern = re.compile('|'.join(combined_patterns.keys()))

    def replace(match):
        return combined_patterns[re.escape(match.group(0))]

    return pattern.sub(replace, text)

def remove_username(text):
    return re.sub('@[^\s]+','',text)

symsp = SymSpell()
symsp.load_dictionary('/kaggle/input/dataprocessing/my_frequency_dictionary.txt', term_index=0, count_index=1, separator=' ')

def correct_spelling(text):
    corrected = symsp.lookup_compound(text, max_edit_distance=2)
    return corrected[0].term if corrected else text

def remove_money(text):
    pattern = r'(?:\$|\£|\€|\₹)?\d+(?:\.\d{1,2})?'
    return re.sub(pattern, '', text)

def fix_negation(sentence):
    for i in range(1, len(sentence)):
        if sentence[i-1] in ['not', "n't"]:
            synsets = wordnet.synsets(sentence[i])
            if synsets:
                w1 = synsets[0]
                antonyms = [l.antonyms()[0].name() for l in w1.lemmas() if l.antonyms()]
                if antonyms:
                    max_dissimilarity = 0
                    for ant in antonyms:
                        ant_synsets = wordnet.synsets(ant)
                        if ant_synsets:
                            similarity = w1.wup_similarity(ant_synsets[0])
                            # Check if similarity is None and handle it
                            if similarity is None:
                                continue  # Skip this comparison
                            dissimilarity = 1 - similarity
                            if dissimilarity > max_dissimilarity:
                                max_dissimilarity = dissimilarity
                                sentence[i] = ant
                    sentence[i-1] = ''

    sentence = [word for word in sentence if word]
    return ' '.join(sentence)  # Return a string instead of a list


STOPWORDS = stopwords.words("english")
STOPWORDS += ['u', 'back', 'one']

def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

def remove_freqwords(text):
    return " ".join([word for word in str(text).split() if word not in FREQWORDS])

def remove_rarewords(text):
    return " ".join([word for word in str(text).split() if word not in RAREWORDS])

def lemmatize_words(text):
    lemmatizer = WordNetLemmatizer()
    wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}
    pos_tagged_text = nltk.pos_tag(text.split())
    return " ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])


tqdm.pandas()

text_preprocessing_pipeline = Pipeline(steps=[
    ('lowercase', FunctionTransformer(lambda x: x.progress_apply(str.lower))),
    ('remove excess whitespace', FunctionTransformer(lambda x: x.progress_apply(remove_excess_space))),
    ('remove_url', FunctionTransformer(lambda x: x.progress_apply(remove_url))),
    ('html_tag_remover', FunctionTransformer(lambda x: x.progress_apply(html_tag_remover))),
    ('money_remover', FunctionTransformer(lambda x: x.progress_apply(remove_money))),
    ('convert_emoticons_and_emojii', FunctionTransformer(lambda x: x.progress_apply(convert_emoticons_and_emojis))),
    ('remove_username', FunctionTransformer(lambda x: x.progress_apply(remove_username))),
    ('replace_slang', FunctionTransformer(lambda x: x.progress_apply(replace_slang))),
    ('correct_spelling', FunctionTransformer(lambda x: x.progress_apply(correct_spelling))),
    ('expand_contractions', FunctionTransformer(lambda x: x.progress_apply(lambda text: contractions.fix(text)))),
    ('fix_negation', FunctionTransformer(lambda x: x.progress_apply(lambda text: "".join(fix_negation(word_tokenize(text)))))),
    ('remove_punctuation', FunctionTransformer(lambda x: x.progress_apply(remove_punctuation))),
    ('normalize_exaggerated_text', FunctionTransformer(lambda x: x.progress_apply(normalize_exaggerated_text))),
    ('remove_stopwords', FunctionTransformer(lambda x: x.progress_apply(remove_stopwords))),
    ('remove_freqwords', FunctionTransformer(lambda x: x.progress_apply(remove_freqwords))),
    ('remove_rarewords', FunctionTransformer(lambda x: x.progress_apply(remove_rarewords))),
    ('lemmatize_words', FunctionTransformer(lambda x: x.progress_apply(lemmatize_words)))
])

In [ ]:
test_df = data_cleaning_pipeline.fit_transform(test_df)
test_df['clean_text'] = text_preprocessing_pipeline.fit_transform(test_df['review_text'])
test_df.head()

In [ ]:
def classify_nps(rating_text):
    rating_text = rating_text.strip()
    if "Rated" in rating_text:
        rating = int(rating_text.split(' ')[1])
        return "Detractor" if rating <= 2 else ("Neutral" if rating == 3 else "Promoter")
    else:
        rating = int(float(rating_text))
        return "Detractor" if rating <= 6 else ("Neutral" if rating <= 8 else "Promoter")



test_df["NPS_category"] = test_df["overall_rating"].apply(classify_nps)
test_df["NPS_category"] = test_df["NPS_category"].map({"Detractor": 0, "Neutral": 1, "Promoter": 2})
test_df.head()

In [ ]:
test_df = test_df.reset_index(drop=True)
test_df

In [ ]:
test_df.dropna(inplace=True)

In [ ]:
df = pd.read_csv("/kaggle/working/cleaned_dataset_v2.csv")
df.head()

In [ ]:
df.dropna(inplace=True)

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(df['clean_text'], df['NPS_category'], test_size=0.25, random_state=42)

tfidf_vectorizer = TfidfVectorizer(lowercase=True)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

class_idfs = []
unique_classes = np.unique(y_train)
for class_idx in unique_classes:
    class_mask = (y_train == class_idx)
    class_docs = X_train[class_mask]
    class_vectorizer = TfidfVectorizer(vocabulary=tfidf_vectorizer.vocabulary_, lowercase=True)
    class_tfidf = class_vectorizer.fit_transform(class_docs)
    class_idf = class_vectorizer.idf_
    class_idfs.append(class_idf)

mean_idf = np.mean(class_idfs, axis=0)

delta_idfs = [class_idf - mean_idf for class_idf in class_idfs]

class_to_index = {label: index for index, label in enumerate(unique_classes)}
X_train_delta_tfidf = []
for i, doc in enumerate(X_train_tfidf):
    class_idx = class_to_index[y_train.iloc[i]]
    delta_idf = delta_idfs[class_idx]
    X_train_delta_tfidf.append(doc.multiply(delta_idf))

X_train_delta_tfidf = vstack(X_train_delta_tfidf)

clf = SGDClassifier(loss='hinge', class_weight='balanced', max_iter=1000, tol=1e-3)
clf.fit(X_train_delta_tfidf, y_train)

In [ ]:
X_test_tfidf = tfidf_vectorizer.transform(test_df['review_text'])
y_test = test_df['NPS_category']


X_test_delta_tfidf = []
for i, doc in enumerate(X_test_tfidf):
    class_idx = class_to_index[y_test.iloc[i]]
    delta_idf = delta_idfs[class_idx]
    X_test_delta_tfidf.append(doc.multiply(delta_idf))

X_test_delta_tfidf = vstack(X_test_delta_tfidf)


y_pred = clf.predict(X_test_delta_tfidf)
print(classification_report(y_test, y_pred))

In [ ]:
# hyperparameter tuning


sgd_clf = SGDClassifier(loss='hinge', class_weight='balanced', max_iter=1000, tol=1e-3)

param_dist = {
    'alpha': loguniform(1e-7, 1e-2),  # Broaden the range a bit to include smaller and larger values
    'learning_rate': ['optimal', 'invscaling', 'adaptive'],  # This is fine as is, covering all the options
    'eta0': loguniform(1e-6, 1e-1),  # Adjust the range for initial learning rate to be broader
    'penalty': ['l2', 'l1', 'elasticnet'],  # Including all relevant penalties
    'l1_ratio': uniform(0, 1),
    'power_t': uniform(0.1, 0.9)  # For 'invscaling' learning rate, exploring a range for power_t could be beneficial
}

random_search = RandomizedSearchCV(sgd_clf, param_distributions=param_dist, n_iter=50,
                                   scoring='f1_weighted', n_jobs=-1, cv=5, random_state=42)


random_search.fit(X_train_delta_tfidf, y_train)

best_params = random_search.best_params_
best_model = random_search.best_estimator_

print("Best Parameters:", best_params)

y_pred = best_model.predict(X_test_delta_tfidf)
print(classification_report(y_test, y_pred))

In [ ]:
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt="d", cmap="viridis", xticklabels=unique_classes, yticklabels=unique_classes)
plt.title('Confusion Matrix')
plt.ylabel('Actual Labels')
plt.xlabel('Predicted Labels')
plt.show()

In [ ]:
precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred)

fig, ax = plt.subplots(1, 1, figsize=(12, 8))
index = np.arange(len(unique_classes))
bar_width = 0.2

p_bar = ax.bar(index, precision, bar_width, label='Precision')
r_bar = ax.bar(index + bar_width, recall, bar_width, label='Recall')
f1_bar = ax.bar(index + 2 * bar_width, f1, bar_width, label='F1-score')

ax.set_xlabel('Classes')
ax.set_ylabel('Scores')
ax.set_title('Precision, Recall, and F1-Score per Class')
ax.set_xticks(index + bar_width)
ax.set_xticklabels(unique_classes, rotation=45)
ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
import numpy as np

# Binarize the output
y_test_binarized = label_binarize(y_test, classes=np.unique(y_test))
n_classes = y_test_binarized.shape[1]


# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
y_score = clf.decision_function(X_test_delta_tfidf)
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_binarized[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Assuming your classes are numbered from 0 to n-1
unique_classes = np.arange(n_classes)

In [ ]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.io as pio

# Set the default renderer to 'iframe' for Jupyter Notebooks or 'notebook' if using JupyterLab
pio.renderers.default = 'iframe'  # or 'notebook' for JupyterLab

# Initialize the plotly figure with a title and a single subplot for the ROC curves
fig = make_subplots(rows=1, cols=1, subplot_titles=['Multiclass ROC Curve'])

# Define a list of colors for the ROC curves
color_palette = [
    '#d62728',  # Muted blue
    '#ff7f0e',  # Safety orange
    '#2ca02c',  # Cooked asparagus green
    '#d62728',  # Brick red
    '#9467bd',  # Muted purple
    '#8c564b',  # Chestnut brown
    '#e377c2',  # Raspberry yogurt pink
    '#7f7f7f',  # Middle gray
    '#bcbd22',  # Curry yellow-green
    '#17becf'   # Blue-teal
]

for i, color in enumerate(color_palette[:len(unique_classes)]):
    fig.add_trace(go.Scatter(x=fpr[i], y=tpr[i], mode='lines',
                             name=f'Class {unique_classes[i]} (AUC = {roc_auc[i]:.2f})',
                             line=dict(color=color, width=2)))

fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', name='Chance',
                         line=dict(color='gray', width=1.5, dash='dot')))

fig.update_layout(
    title='Multiclass ROC Curve',
    title_x=0.5,
    xaxis_title='False Positive Rate',
    yaxis_title='True Positive Rate',
    legend_title='Class AUC',
    legend=dict(y=0.5, traceorder='reversed', font_size=12),
    margin=dict(l=40, r=40, t=85, b=40),
    paper_bgcolor='white',
    plot_bgcolor='white',
    font=dict(size=12, color='black')
)

fig.update_xaxes(range=[0, 1], showgrid=True, gridwidth=1, gridcolor='lightgray')
fig.update_yaxes(range=[0, 1.05], showgrid=True, gridwidth=1, gridcolor='lightgray')

fig.show()

In [ ]:
fig.show("svg")

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.base import clone
import numpy as np
from scipy.sparse import csr_matrix

X_csr = csr_matrix(X_test_delta_tfidf)

# Initialize StratifiedKFold
n_splits = 5
cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Collect F1 scores for each fold and class
f1_scores = []

for train_idx, test_idx in cv.split(X_csr, y_test):
    X_train_fold, X_test_fold = X_csr[train_idx], X_csr[test_idx]
    y_train_fold, y_test_fold = y_test.iloc[train_idx], y_test.iloc[test_idx]

    # Fit and predict
    model = clone(best_model)
    model.fit(X_train_fold, y_train_fold)
    y_pred_fold = model.predict(X_test_fold)

    # Compute F1-scores for each class and append to list
    f1_scores.append(f1_score(y_test_fold, y_pred_fold, average=None))

# Convert F1 scores to a DataFrame
f1_df = pd.DataFrame(f1_scores, columns=np.unique(y_test))

# Melt the DataFrame for seaborn compatibility
f1_melted = f1_df.melt(var_name='Class', value_name='F1 Score')

# Plot
plt.figure(figsize=(12, 6))
sns.violinplot(x='Class', y='F1 Score', data=f1_melted)
plt.title('F1 Score Distribution by Class Across Folds')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

- class 0 (detractors) shows very robust perfromance with a high f1 score and it frequently shows the high f1 score which is a good sign.
- class 1 (passives/neutrals) shows more variability in performance with the lowest performance being 96.5% which is still not too bad. This is due to the fact that there are very few samples in this class.
- class 2 (promoters) shows good performance as well, having lesser variability than class 1 but not as high of performance as class 0. This is stil fine since it is more important to focus on the detractors rather than the promoters as negative word of mouth is far worse and long lasting.

In [ ]:
from sklearn.metrics import cohen_kappa_score, matthews_corrcoef

kappa = cohen_kappa_score(y_test, y_pred)
print(f"Cohen's Kappa: {kappa}")

matthews_corr = matthews_corrcoef(y_test, y_pred)
print(f"Matthew's Correlation Coefficient: {matthews_corr}")

In [ ]:
from sklearn.model_selection import learning_curve

train_sizes, train_scores, valid_scores = learning_curve(clf, X_train_delta_tfidf, y_train, train_sizes=np.linspace(0.1, 1.0, 5), cv=5)

# Calculate mean and std
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
valid_scores_mean = np.mean(valid_scores, axis=1)
valid_scores_std = np.std(valid_scores, axis=1)

# Plot learning curves
plt.figure(figsize=(12, 8))
plt.fill_between(train_sizes, train_scores_mean - train_scores_std, train_scores_mean + train_scores_std, alpha=0.1, color="r")
plt.fill_between(train_sizes, valid_scores_mean - valid_scores_std, valid_scores_mean + valid_scores_std, alpha=0.1, color="g")
plt.plot(train_sizes, train_scores_mean, 'o-', color="r", label="Training score")
plt.plot(train_sizes, valid_scores_mean, 'o-', color="g", label="Cross-validation score")

plt.title("Learning Curves")
plt.xlabel("Training examples")
plt.ylabel("Score")
plt.legend(loc="best")
plt.grid()
plt.show()

In [ ]:
misclassified_idxs = np.where(y_test != y_pred)[0]
for idx in misclassified_idxs[:5]:
    print("Review Text:", test_df["review_text"].iloc[idx], end="\n\n")
    print("Clean Text:", test_df["clean_text"].iloc[idx], end="\n\n")
    print(f"True class: {y_test.iloc[idx]}, Predicted class: {y_pred[idx]}\n")
    print("=====================================================================================================================")

# ASPECT BASED SENTIMENT ANALYSIS

## Data Preparation (topic modelling)

In [ ]:
data_file_path = "/kaggle/working/cleaned_dataset_v2.csv"
df = pd.read_csv(data_file_path)
df.head()

In [ ]:
df.drop(df[df['clean_text'] == ''].index, inplace=True)
df.dropna(inplace=True)
reviews = df['clean_text'].apply(lambda x: x.split())
print("Length is", len(reviews), end="\n\n")
pprint(reviews[:5])

In [ ]:
id2word = corpora.Dictionary(reviews)
corpus = [id2word.doc2bow(i) for i in reviews]
corpus[0] # looking at first entry

In [ ]:
corpus_example = [[(id2word[id], freq) for id, freq in cp] for cp in corpus[:2]]
corpus_example[0][:10] # printing 10 words from corpus

## Building Models (topic modelling)

In [ ]:
num_topics = 10

lda_model = LdaMulticore(corpus=corpus, id2word=id2word,
                     num_topics=num_topics, iterations=400, alpha='symmetric', per_word_topics=True)

pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

## Evaluation and Hyperparameter Tuning (topic modelling)

In [ ]:
c

In [ ]:
from gensim.models.coherencemodel import CoherenceModel
# Perplexity
print('\nPerplexity : ', lda_model.log_perplexity(corpus))

# Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=reviews, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
import gensim

# Build the bigram and trigram models
bigram = gensim.models.Phrases(reviews, min_count=5, threshold=70) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[reviews], threshold=70)

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)


def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

bigram_reviews = make_bigrams(reviews)
trigram_reviews = make_trigrams(reviews)

# Create Dictionary
id2word = corpora.Dictionary(trigram_reviews)

# Create Corpus: Term Document Frequency
corpus = [id2word.doc2bow(text) for text in trigram_reviews]

# Build LDA model
num_topics = 6

lda_model = LdaMulticore(corpus=corpus,
                         id2word=id2word,
                         num_topics=num_topics,
                         iterations=400,
                         eta=0.01,
                         alpha='symmetric',
                         per_word_topics=True)

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis


pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, id2word)
vis

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.LdaMulticore(corpus=corpus, num_topics=num_topics, id2word=dictionary, passes=10, workers=2)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

# Set parameters for coherence values computation
limit = 40; start = 2; step = 6;

# Call the function
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=trigram_reviews, start=start, limit=limit, step=step)

# Show graph
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

# Select the model and print the topics
optimal_model = model_list[coherence_values.index(max(coherence_values))]
model_topics = optimal_model.show_topics(formatted=False)

In [ ]:
from gensim.models.coherencemodel import CoherenceModel
# Perplexity
print('\nPerplexity : ', lda_model.log_perplexity(corpus))

# Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=reviews, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
from gensim.models import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials

space = {
    'n_topics': hp.choice('n_topics', range(2, 12, 2)),
    'alpha': hp.choice('alpha', ['symmetric', 'asymmetric', 0.01, 0.1, 0.5, 1.0]),
    'eta': hp.choice('eta', ['auto', 0.01, 0.1, 0.5, 1.0])
}

def objective(params):
    lda_model = LdaMulticore(corpus=corpus,
                             id2word=id2word,
                             num_topics=params['n_topics'],
                             alpha=params['alpha'],
                             eta=params['eta'],
                             random_state=42,
                             chunksize=100,
                             passes=10,
                             per_word_topics=True)

    coherence_model_lda = CoherenceModel(model=lda_model, texts=reviews, dictionary=id2word, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()

    return {'loss': -coherence_lda, 'status': STATUS_OK}

max_evals = 20
early_stopping_rounds = 3
best_score = None
no_improvement_count = 0

trials = Trials()

for i in range(1, max_evals + 1):
    best = fmin(fn=objective,
                space=space,
                algo=tpe.suggest,
                max_evals=i,
                trials=trials)

    current_best_score = -trials.best_trial['result']['loss']
    if best_score is None or current_best_score > best_score:
        best_score = current_best_score
        no_improvement_count = 0
    else:
        no_improvement_count += 1

    if no_improvement_count >= early_stopping_rounds:
        print(f"Early stopping triggered after {i} evaluations.")
        break

print("Best parameters:", best)

After tuning the best parameters are:

Alpha (document-topic density): 'symmetric'
Eta (word-topic density): 0.01
Number of Topics: 2

(the values given out after tuning are the index)

In [ ]:
lda_model = LdaMulticore(corpus=corpus,
                         id2word=id2word,
                         num_topics=2,  # from n_topics
                         alpha='symmetric',  # from alpha
                         eta=0.01,  # from eta
                         random_state=42,
                         chunksize=100,
                         passes=10,
                         per_word_topics=True)

pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [ ]:
from gensim.models.coherencemodel import CoherenceModel
# Perplexity
print('\nPerplexity : ', lda_model.log_perplexity(corpus))

# Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=reviews, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

## Building Model

## Inference